# Data Cleaning Google Play Store Esteso

In [122]:
import re
import datetime
import pandas as pd

In [123]:
#df=pd.read_csv('/DATA/raw/data_raw_googleplaystore2020.csv')
#non caricato perchè pesante

In [125]:
df=pd.read_csv('DATA/raw/data_raw_googleplaystore2020.csv')

In [126]:
df.columns

Index(['title', 'description', 'minInstalls', 'score', 'ratings', 'reviews',
       'price', 'free', 'currency', 'sale', 'originalPrice', 'offersIAP',
       'inAppProductPrice', 'size', 'androidVersion', 'developer',
       'developerWebsite', 'genreId', 'video', 'contentRating', 'containsAds',
       'released', 'updated'],
      dtype='object')

In [99]:
#drop duplicated apps
df.drop_duplicates('title',inplace=True)

In [100]:
#drop rows with at least 70% of columns missing
df.dropna(thresh=0.3*len(df.columns),inplace=True)

In [101]:
#df.dtypes
df.isna().sum()

title                    0
description              0
minInstalls              0
score                   88
ratings               1208
reviews                 88
price                    4
free                     4
currency                 4
sale                     0
originalPrice        17451
offersIAP                0
inAppProductPrice    11594
size                     0
androidVersion          47
developer                0
developerWebsite      3800
genreId                  0
video                13150
contentRating            0
containsAds              0
released               175
updated                 91
dtype: int64

In [102]:
#create a variable measuring length of the description
df['lenDescription']=df['description'].str.len()
df.drop(columns='description',axis=1,inplace=True)

In [103]:
#drop rows with nan in the price field, as such rows are missing other important fields
df.dropna(subset=['price'],inplace=True)
#drop rows with app price > 250 - scammy apps
df.drop(df[df.price>250].index,inplace=True)

In [104]:
#rename the free column and relabel values to improve clarity when graphing
df.rename(columns={'free':'type'},inplace=True)
df["type"].replace({True:'Free',False:'Paid'},inplace=True)

In [105]:
#very few obs with currency different than USD; free apps with XXX under currency are all free with in app product price in $ 
#keep the column currency just to remind that everything is in usd
df['currency']='USD'

In [106]:
#sale - relabel values to improve clarity when graphing
df["sale"].replace({True:'Sale',False:'Not Sale'},inplace=True)

In [107]:
#original price - drop and create new feature measuring the % of the price drop during sale
df['saleDiscountPerc'] = df.apply(lambda row: abs(((row.price/row.originalPrice)-1)*100), axis=1)
df['saleDiscountPerc'].fillna(0,inplace=True)
df.drop(columns='originalPrice',axis=1,inplace=True)

In [108]:
#rename the offersIAP column and relabel values to improve clarity when graphing
df.rename(columns={'offersIAP':'inAppPurchase'},inplace=True)
df["inAppPurchase"].replace({True:'Yes',False:'No'},inplace=True)

In [109]:
#drop inAppProductPrice and create three new features: max,min,mean, range of in app prices
df['inAppProductPrice'].fillna(0,inplace=True)
df['inAppProductPrice'] = df['inAppProductPrice'].apply(lambda row: str(row).replace('$', '') if '$' in str(row) else str(row))
df['inAppProductPrice'] = df['inAppProductPrice'].apply(lambda row: str(row).replace('per item', '') if 'per item' in str(row) else str(row))
df['inAppProductPrice'] = df['inAppProductPrice'].apply(lambda row: str(row).replace(' ', '') if ' ' in str(row) else str(row))
df['inAppProductPrice']=df['inAppProductPrice'].apply(lambda row: str(row).split("-"))

df['inAppPriceMin']=df['inAppProductPrice'].apply(lambda row: row[0])
df['inAppPriceMax']=df['inAppProductPrice'].apply(lambda row: row[-1])
df['inAppPriceMin']=pd.to_numeric(df['inAppPriceMin'])
df.loc[df['inAppPriceMax']=='1,024.00','inAppPriceMax']='1024'
df['inAppPriceMax']=pd.to_numeric(df['inAppPriceMax'])
df['inAppPriceRange']=df['inAppPriceMax']-df['inAppPriceMin']
df['inAppPriceMean']=df['inAppPriceMax']-df['inAppPriceMin']/2
df.drop(columns='inAppProductPrice',inplace=True)

In [110]:
#Uniform size in MB
df['size'] = df['size'].apply(lambda row: str(row).replace('Varies with device', '-1'))
df['size'] = df['size'].apply(lambda row: re.sub('[M,]', '', row))
df['size'] = df['size'].apply(lambda row: (float(str(row).replace('k', '')) / 1000) if 'k' in str(row) else row)
df['size'] = df['size'].apply(lambda row: (float(str(row).replace('G', ''))* 1024) if 'G' in str(row) else row)
df['size'] = pd.to_numeric(df['size'])

In [111]:
# transforms android version column to numeric
df['androidVersion'] = df['androidVersion'].apply(lambda row: re.sub('[^0-9]', '', str(row)))
df['androidVersion'] = df['androidVersion'].apply(lambda row: str(row).replace('Varies', '-1'))
df['androidVersion'].fillna('-1',inplace=True)
#df['androidVersion'] = pd.to_numeric(df['androidVersion'])

In [112]:
#transform the column video in a bool
df['videoAvailable']=df.video.notna()
df['videoAvailable'].replace({True:'Yes',False:'No'},inplace=True)
df.drop(columns='video',inplace=True)

In [113]:
#Content Rating check distribution of values - only one app for 18+ and it is a game - so moved to 17+
df.contentRating.value_counts()
df['contentRating'].replace({'Adults only 18+':'Mature 17+',}, inplace = True)

In [114]:
#relabel ContainAds values to improve clarity when graphing
df['containsAds'].replace({True:'Yes',False:'No'},inplace=True)

In [115]:
#Transform the Released and Updated fields in a datetime format. Create a new feature mesuring the age of the app
scraping_date= datetime.datetime(2020, 12, 2)
df['released'] = pd.to_datetime(df['released'], infer_datetime_format = True)
df['appDaysSinceReleased']=(scraping_date - df.released).astype('timedelta64[D]')
df['appDaysSinceReleased'].fillna(df.appDaysSinceReleased.median(),inplace=True)
df.drop(columns='released',inplace=True)
df['updated'] = pd.to_datetime(df['updated'], infer_datetime_format = True)
df['appDaysSinceUpdated']=(scraping_date - df.updated).astype('timedelta64[D]')
df['appDaysSinceUpdated'].fillna(df.appDaysSinceReleased.median(),inplace=True)

In [116]:
#The column rating is how many times a score is given. The column reviews is how many reviews has been given.
#df[df.ratings<df.reviews]
df[['ratings','reviews']].corr()
#the variables are heavily correlated since they measure the same underlying phenomenon
df[['ratings','reviews']].isna().sum()
#drop rating which has a higher number of nan
df.drop(columns='ratings',inplace=True)

In [117]:
#Reviews
#the rows where the number of reviews is missing are also missing the score value. this probably means that the missing values are 
#in fact apps that have not been rated/reviewed. Apps w/o a score are markes as Not Rated with number of reviews set to 0.
df.reviews.fillna(0,inplace=True)


In [118]:
#Score - the rating must be between 1 and 5
#No scores above 5
df['score']=round(df['score'])
df[df['score']>5]
#Some "0" scores
set(df[df['score']<1])
#rows with "0" scores have also 0 reviews - meaning they have not been rated
set(df.loc[df['score']==0,'reviews'])
df['score'].replace({0:'NR'}, inplace = True)
df.score.fillna('NR',inplace=True)

In [119]:
#GenreId rename as category-group all games together
df['genreId']=df['genreId'].apply(lambda row: row.split("_")[0] if row.startswith('GAME_') else row)

In [120]:
df.to_csv('/Users/Debora/Desktop/Projects/CLEAN_NLP/DATA/pre-processed/clean_app_data2020.csv',index=False)
